In [2]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [3]:
from django.db.models import Q, Max, F
from collections import Counter

In [4]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        queryset = queryset.filter(q_objects)
    
    return queryset

In [5]:

user_keywords = ['烏克蘭', '俄羅斯']  # Example keywords
cond = 'and'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [6]:
for news in queryset:
    print(news.title)

美國最新全球威脅評估 烏俄都可能延長戰爭
雙方破冰重要角色 俄羅斯特使曝：俄美最快下週再度接觸


烏克蘭基輔遭飛彈攻擊 多人受傷
烏軍稱俘虜兩名中國公民 美國國務院：令人不安
烏證實攻俄貝爾哥羅德
烏克蘭檢方：1男受命莫斯科策劃暗殺遭到拘留
美俄商恢復使館運作 澤倫斯基：至少155中國公民為俄作戰
英法召開首場援烏志願者聯盟國防部長會議 30國與會
傳陸兵為俄淪炮灰 真正目的劍指台灣？大陸中立背後恐是「軍事實習」
美俄換囚 捐款援烏芭蕾舞者獲釋
英日首相通話  英相感謝日本支持援烏志願者聯盟
英國考慮向烏克蘭派兵5年！協助重建軍力防止俄羅斯再進犯
中國北韓與俄軍事合作 美軍司令：升高印太威脅
美參院通過！退將凱恩出任美軍參謀首長聯席會議主席
美烏華府會商礦產協議疑「獅子大開口」 傳氣氛對抗恐難突破
會談逾4小時…普亭會美特使沒結果 川普：俄須快點行動
川普：俄烏停戰談判可能進展順利 但需妥協或閉嘴
俄烏停火慢 川普促俄加快
凱洛格：仿戰後德國 畫分東西烏


In [7]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [8]:
news_items

<QuerySet [{'category': '全球', 'title': '美國最新全球威脅評估 烏俄都可能延長戰爭', 'link': 'https://udn.com/news/story/122663/8634467?from=udn-catebreaknews_ch2', 'photo_link': 'https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2025/03/26/realtime/31726018.jpg&s=Y&x=102&y=0&sw=1077&sh=719&exp=3600&w=300'}, {'category': '全球', 'title': '雙方破冰重要角色 俄羅斯特使曝：俄美最快下週再度接觸', 'link': 'https://udn.com/news/story/122663/8656471?from=udn-catelistnews_ch2', 'photo_link': 'https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2025/04/06/98/31799814.jpg&x=&y=&sw=&sh=&exp=3600&w=300'}, {'category': '全球', 'title': '烏克蘭基輔遭飛彈攻擊 多人受傷', 'link': 'https://udn.com/news/story/122663/8655990?from=udn-catelistnews_ch2', 'photo_link': 'https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/wj/realtime/2025/04/07/31800443.jpg&s=Y&x=0&y=9&sw=1280&sh=852&exp=3600&w=300'}]>

In [9]:
dates = list(queryset.values_list('date', flat=True))

In [10]:
NewsData.objects.all()

<QuerySet [<NewsData: 2025-03-10: 美國擬藉3/11沙烏地會談 評估烏克蘭結束戰爭意願>, <NewsData: 2025-03-10: 《富爸爸》作者清崎警告史上最大股市崩盤將至 建議買「這些資產」>, <NewsData: 2025-03-10: 川普上任後北韓首射彈道飛彈 可能針對聯合軍演>, <NewsData: 2025-03-10: 為躲無人機攻擊 俄軍用上驢和馬、烏軍推手推車>, <NewsData: 2025-03-10: 「走到哪兒扎到哪兒」 醫界人大代表在兩會針灸扎了300多針>, <NewsData: 2025-03-10: 李強整治「內捲」競爭！大疆、美的等陸企加入行列 嚴禁形式主義加班>, <NewsData: 2025-03-10: 陸消費價格漲幅目標「2%左右」 官媒指實事求是：有能力保持物價在合理區間>, <NewsData: 2025-03-10: 北大台研院長談國際形勢：川普對台海可能表現出「交易論」、續打台灣牌>, <NewsData: 2025-03-10: 竹聯幫仁堂大哥「麥可」許瑞弘被拘提 涉組織犯罪案>, <NewsData: 2025-03-10: 酒駕與夫口角 女子鶯歌連撞7機車公共危險送辦>, <NewsData: 2025-03-10: 傳竹聯幫仁堂大老「麥克」遭逮 士檢證實：案件偵辦中>, <NewsData: 2025-03-10: 中部警界、海巡涉私菸案貪凟洩密 彰檢搜索38處約詢29人>, <NewsData: 2025-03-10: 新光三越氣爆櫃位移動 由原長榮桂冠轉台中世貿二館營運>, <NewsData: 2025-03-10: 海軍敦睦艦隊抵臺南安平商港 3/9-10開放登艦參觀>, <NewsData: 2025-03-10: 台南鹿耳門媽文物館動土大典 媽祖信仰傳承新里程碑>, <NewsData: 2025-03-10: 雲林北港地王每坪300萬元？星巴克現址土地傳出售 房仲疑誤傳>, <NewsData: 2025-03-10: 北醫大與GSK台灣分公司簽訂MOU 強化醫藥人才培育>, <NewsData: 2025-03-10: 43歲的她「陰道痙攣」無法行房 人工受孕才順利得子>, <NewsData: 2025-03-10: 文組讀研究所更有加薪機會